In [ ]:
import sqlite3
import datetime
import os

class Expense:
    def __init__(self, category, amount, date):
        self.category = category
        self.amount = amount
        self.date = date

class ExpenseTracker:
    def __init__(self, conn):
        self.conn = conn
        self.create_table()

    def create_table(self):
        cursor = self.conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS expenses
                          (id INTEGER PRIMARY KEY, category TEXT, amount REAL, date TEXT)''')
        self.conn.commit()

    def add_expense(self, expense):
        cursor = self.conn.cursor()
        cursor.execute("INSERT INTO expenses (category, amount, date) VALUES (?, ?, ?)", (expense.category, expense.amount, expense.date.strftime('%Y-%m-%d')))
        self.conn.commit()

    def total_expenses_per_month(self, month):
        cursor = self.conn.cursor()
        cursor.execute("SELECT SUM(amount) FROM expenses WHERE strftime('%m', date) = ?", (str(month).zfill(2),))
        result = cursor.fetchone()
        return result[0] if result[0] else 0

    def total_expenses_per_category(self, month):
        cursor = self.conn.cursor()
        cursor.execute("SELECT category, SUM(amount) FROM expenses WHERE strftime('%m', date) = ? GROUP BY category", (str(month).zfill(2),))
        rows = cursor.fetchall()
        category_expenses = {}
        for row in rows:
            category_expenses[row[0]] = row[1]
        return category_expenses

    def annual_average_expenses_per_category(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT category, AVG(amount) FROM expenses GROUP BY category")
        rows = cursor.fetchall()
        annual_averages = {}
        for row in rows:
            annual_averages[row[0]] = row[1]
        return annual_averages

    def generate_monthly_report(self, month):
        total_monthly_expenses = self.total_expenses_per_month(month)
        category_expenses = self.total_expenses_per_category(month)
        annual_averages = self.annual_average_expenses_per_category()
        print("------------------")
        print(f"Monthly Report for {datetime.datetime.now().strftime('%Y')}-{month}")
        print("Total Monthly Expenses:", total_monthly_expenses)
        print()

        for category, amount in category_expenses.items():
            annual_average = annual_averages.get(category, 0)
            print("------------------")
            print(f"Category: {category}")
            print("Monthly Expenses:", amount)
            print("Annual Average Expenses:", annual_average)
            if annual_average:
                if amount > annual_average:
                    print("Your expenses for this month are higher than the annual average.")
                elif amount < annual_average:
                    print("Your expenses for this month are lower than the annual average.")
                else:
                    print("Your expenses for this month are equal to the annual average.")
            percentage = (amount / total_monthly_expenses) * 100 if total_monthly_expenses != 0 else 0
            print("Percentage of Total Monthly Expenses:", round(percentage, 2), "%")
            print("------------------")
            print()


In [ ]:
def menu():
    print("--Expense Tracker Menu--")
    print("1. Add Expense")
    print("2. Generate Monthly Report")
    print("3. Exit")

def get_month_input():
    while True:
        month_input = input("Enter month (1-12): ")
        try:
            month = int(month_input)
            if 1 <= month <= 12:
                return month
            else:
                print("Invalid month. Please enter a number between 1 and 12.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

def get_day_input():
    while True:
        day_input = input("Enter day (1-31): ")
        try:
            day = int(day_input)
            if 1 <= day <= 31:
                return day
            else:
                print("Invalid day. Please enter a number between 1 and 31.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

def main():
    if not os.path.exists('expenses.db'):
        conn = sqlite3.connect('expenses.db')
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS expenses
                          (id INTEGER PRIMARY KEY, category TEXT, amount REAL, date TEXT)''')
        conn.commit()
        conn.close()

    conn = sqlite3.connect('expenses.db')
    expense_tracker = ExpenseTracker(conn)

    while True:
        menu()
        choice = input("Enter your choice: ")

        if choice not in {'1', '2', '3'}:
            print("Invalid choice. Please enter a number between 1 and 3.")
            continue

        if choice == '1':
            category = input("Enter expense category: ")
            while True:
                amount_input = input("Enter expense amount: ")
                try:
                    amount = float(amount_input)
                    break
                except ValueError:
                    print("Invalid input. Please enter a valid number.")

            month = get_month_input()
            day = get_day_input()
            date = datetime.datetime(datetime.datetime.now().year, month, day)
            expense = Expense(category, amount, date)
            expense_tracker.add_expense(expense)
            print("Expense added successfully.")

        elif choice == '2':
            month = get_month_input()
            expense_tracker.generate_monthly_report(month)

        elif choice == '3':
            conn.close()
            print("Exiting...")
            break

if __name__ == "__main__":
    main()

--Expense Tracker Menu--
1. Add Expense
2. Generate Monthly Report
3. Exit
Enter your choice: 2
Enter month (1-12): 6
------------------
Monthly Report for 2024-6
Total Monthly Expenses: 6100.0

------------------
Category: food
Monthly Expenses: 1000.0
Annual Average Expenses: 566.6666666666666
Your expenses for this month are higher than the annual average.
Percentage of Total Monthly Expenses: 16.39 %
------------------

------------------
Category: rent
Monthly Expenses: 5100.0
Annual Average Expenses: 2325.0
Your expenses for this month are higher than the annual average.
Percentage of Total Monthly Expenses: 83.61 %
------------------

--Expense Tracker Menu--
1. Add Expense
2. Generate Monthly Report
3. Exit
Enter your choice: 3
Exiting...
